# Practicumopdracht 1
*Gemaakt door Yaika van der Knokke (24149128) en Tim Postma (24144738)*

In deze opdracht wordt gekeken naar een pastaria: *La Cucina della Nonna*. Vanwege wettelijke beperkingen worden veel mensen weggestuurd voordat zij hier bediend kunnen worden. De pastaria wil deze situatie graag verbeteren en een groter aantal klanten serveren. Hiervoor zijn drie oplossingen bedacht, en in deze opdracht wordt gekeken, met behulp van een simulatie, welke oplossing het beste past bij de pastaria. De beste wordt gedefinieerd als de oplossing die het snelst wordt terugverdiend.

### Toelichting simulaties
Hieronder worden de diverse situaties van de pastaria gesimuleerd. Om een zo'n goed mogelijk resultaat te krijgen zijn de simulaties met behulp van Monte Carlo 200 keer uitgevoerd. Door genoeg simulaties te hebben kan gebruik gemaakt worden van de Centrale Limietstelling.
De resultaten van elke individuele simulatie wordt opgeslagen in een DataFrame.

In deze dataframe word onderandere het volgende opgeslagen:

- Aantal kopers die aankomen bij de pastaria.
- Aantal eters die aankomen bij de pastaria.
- Aantal kopers die niet bediend kunnen worden en teleurgesteld naar huis gaan.
- Aantal eters die niet bediend kunnen worden en teleurgesteld naar huis gaan.
- De totale omzet van de pastaria.

### Aannames
Hierbij zijn de volgende aannames gemaakt:

- Klanten komen altijd alleen
- Er is geen sprakte van balking of reneging (= Ze gaan niet-niet in de rij staan of verlaten de wachtrij)
- Na sluitingstijd worden de klanten die al bij de pastaria in de wachtrij staan nog geholpen

In [1]:
# importeren packages
import salabim as sim
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from collections import Counter
import statsmodels.stats.weightstats
import statsmodels.stats.api as sms
import random

## Huidige Situatie

In de <u>Huidige Situatie</u> worden er maximaal 2 eters tegelijkertijd bediend, 4 eters kunnen in de wachtrij, 1 koper wordt tegelijkertijd bediend en er mogen 3 kopers tegelijkertijd wachten. Verder is de pastaria open voor 6 uur per dag.

In [2]:
# Lijsten van resultaten van de losse simulaties
kopers = []
eters = []
kopers_afgehaakt = []
eters_afgehaakt = []
omzet_kopers = []
omzet_eters = []
omzet = []
model = []

# Genereer klanten
class KlantenGenerator(sim.Component):
    def process(self):
        while self.env.now() < 6*60:   # Alleen klanten genereren zolang de pastaria open is
            if random.uniform(0,1) <= 0.4:  # 40% eters
                Eter()
                stats['eters'] += 1
            else:                           # 60% kopers
                Koper()
                stats['kopers'] += 1
            # Gemiddeld 1.5 klant per minuut
            self.hold(sim.Exponential(2/3).sample())

# Eter die aangekomen is (die hopelijk) in de wachtrij mag gaan staan en daarna eten
class Eter(sim.Component):
    def process(self):
        if Eters_b.requesters().length() < 4:  # Max wachtrij = 4
            self.request(Eters_b)
            self.hold(sim.Normal(9, 1).sample())  # Bedieningstijd
            omzet = sim.Bounded(sim.Normal(20, 2), 0).sample() # Omzet van een eter, volgt normaal verdeling, omzet kan niet negatief zijn.
            stats['opbrengst_eters'] += omzet
        else:
            stats['eters_afgehaakt'] += 1 # Eter die niet in de wachtrij plaats kon nemen, en weer teleurgesteld naar huis moet.
            self.cancel()

# Koper die aangekomen is (die hopelijk) in de wachtrij mag gaan staan en daarna eten meenemen naar huis!
class Koper(sim.Component):
    def process(self):
        if Kopers_b.requesters().length() < 3:  # Max wachtrij = 3
            self.request(Kopers_b)
            self.hold(sim.Exponential(2).sample())  # Bedieningstijd
            omzet = sim.Uniform(3, 7).sample() # Omzet van een koper, volgt uniforme verdeling.
            stats['opbrengst_kopers'] += omzet
        else:
            stats['kopers_afgehaakt'] += 1 # Teleurgestelde koper die naar huis moet
            self.cancel()

# Uitvoeren van 200 simulaties (Monte Carlo)
for i in range(200):
    env = sim.Environment(trace=False, random_seed="*")
    stats = {"kopers": 0, "eters": 0, "kopers_afgehaakt": 0, "eters_afgehaakt": 0,"opbrengst_eters": 0, "opbrengst_kopers": 0}

    KlantenGenerator()
    Eters_b = sim.Resource('Eters bediening', capacity=2)
    Kopers_b = sim.Resource('Kopers bediening', capacity=1)

    # Uitvoeren v/d simulatie
    env.run()

    # Resultaten opslaan
    kopers.append(stats['kopers'])
    eters.append(stats['eters'])
    kopers_afgehaakt.append(stats['kopers_afgehaakt'])
    eters_afgehaakt.append(stats['eters_afgehaakt'])
    omzet_kopers.append(stats['opbrengst_kopers'])
    omzet_eters.append(stats['opbrengst_eters'])
    omzet.append(stats['opbrengst_kopers'] + stats['opbrengst_eters'])
    model.append("Pastaria huidig")

df1 = pd.DataFrame([kopers, eters, kopers_afgehaakt, eters_afgehaakt, omzet_kopers, omzet_eters, omzet, model]).T
df1 = df1.rename({0: 'Kopers', 1: 'Eters', 2: 'Kopers_afgehaakt', 3: 'Eters_afgehaakt', 4: 'Omzet_kopers', 5: 'Omzet_eters', 6: 'Omzet',7: 'Model'}, axis='columns')

# Eerste rijen van de dataframe laten zien.
display(df1.head())


,Kopers,Eters,Kopers_afgehaakt,Eters_afgehaakt,Omzet_kopers,Omzet_eters,Omzet,Model
0,336,201,165,117,859.950798,1731.940015,2591.890812,Pastaria huidig
1,324,219,121,134,1059.402868,1721.966841,2781.369709,Pastaria huidig
2,316,218,134,134,923.908743,1685.834626,2609.74337,Pastaria huidig
3,332,218,166,134,823.727921,1687.635948,2511.363869,Pastaria huidig
4,330,255,152,172,874.58048,1668.180215,2542.760695,Pastaria huidig


## Variant A

In <u>Variant A</u> worden er net zoals in de <u>Huidige Situatie</u> maximaal 2 eters tegelijkertijd bediend, 4 eters kunnen in de wachtrij, 1 koper wordt tegelijkertijd bediend en er mogen 3 kopers tegelijkertijd wachten. Het verschil hiermee is dat de pastaria nu 8 uur per dag open is. Dit is een investering die eenmalig `€60.000`kost.

In [3]:
# Lijsten van resultaten van de losse simulaties
kopers = []
eters = []
kopers_afgehaakt = []
eters_afgehaakt = []
omzet_kopers = []
omzet_eters = []
omzet = []
model = []

# Genereer klanten
class KlantenGenerator(sim.Component):
    def process(self):
        while self.env.now() < 8*60:   # Alleen klanten genereren zolang de pastaria open is
            if random.uniform(0,1) <= 0.4:  # 40% eters
                Eter()
                stats['eters'] += 1
            else:                           # 60% kopers
                Koper()
                stats['kopers'] += 1
            # Gemiddeld 1.5 klant per minuut
            self.hold(sim.Exponential(2/3).sample())

# Eter die aangekomen is (die hopelijk) in de wachtrij mag gaan staan en daarna eten
class Eter(sim.Component):
    def process(self):
        if Eters_b.requesters().length() < 4:  # Max wachtrij = 4
            self.request(Eters_b)
            self.hold(sim.Normal(9, 1).sample())  # Bedieningstijd
            omzet = sim.Bounded(sim.Normal(20, 2), 0).sample() # Omzet van een eter, volgt normaal verdeling, omzet kan niet negatief zijn.
            stats['opbrengst_eters'] += omzet
        else:
            stats['eters_afgehaakt'] += 1 # Eter die niet in de wachtrij plaats kon nemen, en weer teleurgesteld naar huis moet.
            self.cancel()

# Koper die aangekomen is (die hopelijk) in de wachtrij mag gaan staan en daarna eten meenemen naar huis!
class Koper(sim.Component):
    def process(self):
        if Kopers_b.requesters().length() < 3:  # Max wachtrij = 3
            self.request(Kopers_b)
            self.hold(sim.Exponential(2).sample())  # Bedieningstijd
            omzet = sim.Uniform(3, 7).sample() # Omzet van een koper, volgt uniforme verdeling.
            stats['opbrengst_kopers'] += omzet
        else:
            stats['kopers_afgehaakt'] += 1 # Teleurgestelde koper die naar huis moet
            self.cancel()

# Uitvoeren van 200 simulaties (Monte Carlo)
for i in range(200):
    env = sim.Environment(trace=False, random_seed="*")
    stats = {"kopers": 0, "eters": 0, "kopers_afgehaakt": 0, "eters_afgehaakt": 0,"opbrengst_eters": 0, "opbrengst_kopers": 0}

    KlantenGenerator()
    Eters_b = sim.Resource('Eters bediening', capacity=2)
    Kopers_b = sim.Resource('Kopers bediening', capacity=1)

    # Uitvoeren v/d simulatie
    env.run()

    # Resultaten opslaan
    kopers.append(stats['kopers'])
    eters.append(stats['eters'])
    kopers_afgehaakt.append(stats['kopers_afgehaakt'])
    eters_afgehaakt.append(stats['eters_afgehaakt'])
    omzet_kopers.append(stats['opbrengst_kopers'])
    omzet_eters.append(stats['opbrengst_eters'])
    omzet.append(stats['opbrengst_kopers'] + stats['opbrengst_eters'])
    model.append("VariantA")

df2 = pd.DataFrame([kopers, eters, kopers_afgehaakt, eters_afgehaakt, omzet_kopers, omzet_eters, omzet, model]).T
df2 = df2.rename({0: 'Kopers', 1: 'Eters', 2: 'Kopers_afgehaakt', 3: 'Eters_afgehaakt', 4: 'Omzet_kopers', 5: 'Omzet_eters', 6: 'Omzet',7: 'Model'}, axis='columns')

# Eerste rijen van de dataframe laten zien.
display(df2.head())

,Kopers,Eters,Kopers_afgehaakt,Eters_afgehaakt,Omzet_kopers,Omzet_eters,Omzet,Model
0,419,277,187,165,1154.818991,2228.430687,3383.249677,VariantA
1,434,291,210,180,1129.369353,2257.480599,3386.849952,VariantA
2,440,277,210,164,1160.930931,2271.650225,3432.581156,VariantA
3,427,297,203,185,1113.987169,2250.896286,3364.883454,VariantA
4,428,280,163,168,1322.014423,2232.429979,3554.444402,VariantA


## Variant B

In <u>Variant B</u> worden er maximaal 4 eters tegelijkertijd bediend, 2 eters kunnen in de wachtrij, 4 kopers worden tegelijkertijd bediend en er mogen 0 kopers wachten. De pastaria 6 uur per dag open. Dit is een investering die eenmalig `€120.000`kost.

In [4]:
# Lijsten van resultaten van de losse simulaties
kopers = []
eters = []
kopers_afgehaakt = []
eters_afgehaakt = []
omzet_kopers = []
omzet_eters = []
omzet = []
model = []

# Genereer klanten
class KlantenGenerator(sim.Component):
    def process(self):
        while self.env.now() < 6*60:   # Alleen klanten genereren zolang de pastaria open is
            if random.uniform(0,1) <= 0.4:  # 40% eters
                Eter()
                stats['eters'] += 1
            else:                           # 60% kopers
                Koper()
                stats['kopers'] += 1
            # Gemiddeld 1.5 klant per minuut
            self.hold(sim.Exponential(2/3).sample())

# Eter die aangekomen is (die hopelijk) in de wachtrij mag gaan staan en daarna eten
class Eter(sim.Component):
    def process(self):
        if Eters_b.requesters().length() < 2:  # Max wachtrij = 2
            self.request(Eters_b)
            self.hold(sim.Normal(9, 1).sample())  # Bedieningstijd
            omzet = sim.Bounded(sim.Normal(20, 2), 0).sample() # Omzet van een eter, volgt normaal verdeling, omzet kan niet negatief zijn.
            stats['opbrengst_eters'] += omzet
        else:
            stats['eters_afgehaakt'] += 1 # Eter die niet in de wachtrij plaats kon nemen, en weer teleurgesteld naar huis moet.
            self.cancel()

# Koper die aangekomen is (die hopelijk) in de wachtrij mag gaan staan en daarna eten meenemen naar huis!
class Koper(sim.Component):
    def process(self):
        if Kopers_b.claimers().length() < 4:  # Max 4 kopers tegelijk bediend
            self.request(Kopers_b)
            self.hold(sim.Exponential(2).sample())  # Bedieningstijd
            omzet = sim.Uniform(3, 7).sample() # Omzet van een koper, volgt uniforme verdeling.
            stats['opbrengst_kopers'] += omzet
        else:
            stats['kopers_afgehaakt'] += 1 # Teleurgestelde koper die naar huis moet
            self.cancel()

# Uitvoeren van 200 simulaties (Monte Carlo)
for i in range(200):
    env = sim.Environment(trace=False, random_seed="*")
    stats = {"kopers": 0, "eters": 0, "kopers_afgehaakt": 0, "eters_afgehaakt": 0,"opbrengst_eters": 0, "opbrengst_kopers": 0}

    KlantenGenerator()
    Eters_b = sim.Resource('Eters bediening', capacity=4)
    Kopers_b = sim.Resource('Kopers bediening', capacity=4)

    # Uitvoeren v/d simulatie
    env.run()

    # Resultaten opslaan
    kopers.append(stats['kopers'])
    eters.append(stats['eters'])
    kopers_afgehaakt.append(stats['kopers_afgehaakt'])
    eters_afgehaakt.append(stats['eters_afgehaakt'])
    omzet_kopers.append(stats['opbrengst_kopers'])
    omzet_eters.append(stats['opbrengst_eters'])
    omzet.append(stats['opbrengst_kopers'] + stats['opbrengst_eters'])
    model.append("VariantB")

df3 = pd.DataFrame([kopers, eters, kopers_afgehaakt, eters_afgehaakt, omzet_kopers, omzet_eters, omzet, model]).T
df3 = df3.rename({0: 'Kopers', 1: 'Eters', 2: 'Kopers_afgehaakt', 3: 'Eters_afgehaakt', 4: 'Omzet_kopers', 5: 'Omzet_eters', 6: 'Omzet',7: 'Model'}, axis='columns')

# Eerste rijen van de dataframe laten zien.
display(df3.head())

,Kopers,Eters,Kopers_afgehaakt,Eters_afgehaakt,Omzet_kopers,Omzet_eters,Omzet,Model
0,335,215,35,65,1507.041627,3009.564853,4516.60648,VariantB
1,312,208,14,62,1479.865836,2933.655609,4413.521445,VariantB
2,315,209,12,61,1546.392222,2969.611983,4516.004205,VariantB
3,333,216,18,64,1574.21898,3045.111008,4619.329989,VariantB
4,343,208,21,55,1630.151673,3015.790307,4645.94198,VariantB


## Variant C

In <u>Variant C</u> worden er maximaal 6 eters tegelijkertijd bediend, 0 eters kunnen in de wachtrij, 2 kopers worden tegelijkertijd bediend en er mogen 2 kopers wachten. De pastaria 6 uur per dag open. Dit is een investering die eenmalig `€150.000`kost.

In [5]:
# Lijsten van resultaten van de losse simulaties
kopers = []
eters = []
kopers_afgehaakt = []
eters_afgehaakt = []
omzet_kopers = []
omzet_eters = []
omzet = []
model = []

# Genereer klanten
class KlantenGenerator(sim.Component):
    def process(self):
        while self.env.now() < 6*60:   # Alleen klanten genereren zolang de pastaria open is
            if random.uniform(0,1) <= 0.4:  # 40% eters
                Eter()
                stats['eters'] += 1
            else:                           # 60% kopers
                Koper()
                stats['kopers'] += 1
            # Gemiddeld 1.5 klant per minuut
            self.hold(sim.Exponential(2/3).sample())

# Eter die aangekomen is (die hopelijk) in de wachtrij mag gaan staan en daarna eten
class Eter(sim.Component):
    def process(self):
        if Eters_b.claimers().length() < 6:  # Maximaal 6 eters tegelijkertijd bediend
            self.request(Eters_b)
            self.hold(sim.Normal(9, 1).sample())  # Bedieningstijd
            omzet = sim.Bounded(sim.Normal(20, 2), 0).sample() # Omzet van een eter, volgt normaal verdeling, omzet kan niet negatief zijn.
            stats['opbrengst_eters'] += omzet
        else:
            stats['eters_afgehaakt'] += 1 # Eter die niet in de wachtrij plaats kon nemen, en weer teleurgesteld naar huis moet.
            self.cancel()

# Koper die aangekomen is (die hopelijk) in de wachtrij mag gaan staan en daarna eten meenemen naar huis!
class Koper(sim.Component):
    def process(self):
        if Kopers_b.requesters().length() < 2:  # Wachtrij is maximaal 2 kopers
            self.request(Kopers_b)
            self.hold(sim.Exponential(2).sample())  # Bedieningstijd
            omzet = sim.Uniform(3, 7).sample() # Omzet van een koper, volgt uniforme verdeling.
            stats['opbrengst_kopers'] += omzet
        else:
            stats['kopers_afgehaakt'] += 1 # Teleurgestelde koper die naar huis moet
            self.cancel()

# Uitvoeren van 200 simulaties (Monte Carlo)
for i in range(200):
    env = sim.Environment(trace=False, random_seed="*")
    stats = {"kopers": 0, "eters": 0, "kopers_afgehaakt": 0, "eters_afgehaakt": 0,"opbrengst_eters": 0, "opbrengst_kopers": 0}

    KlantenGenerator()
    Eters_b = sim.Resource('Eters bediening', capacity=6)
    Kopers_b = sim.Resource('Kopers bediening', capacity=2)

    # Uitvoeren v/d simulatie
    env.run()

    # Resultaten opslaan
    kopers.append(stats['kopers'])
    eters.append(stats['eters'])
    kopers_afgehaakt.append(stats['kopers_afgehaakt'])
    eters_afgehaakt.append(stats['eters_afgehaakt'])
    omzet_kopers.append(stats['opbrengst_kopers'])
    omzet_eters.append(stats['opbrengst_eters'])
    omzet.append(stats['opbrengst_kopers'] + stats['opbrengst_eters'])
    model.append("VariantC")

df4 = pd.DataFrame([kopers, eters, kopers_afgehaakt, eters_afgehaakt, omzet_kopers, omzet_eters, omzet, model]).T
df4 = df4.rename({0: 'Kopers', 1: 'Eters', 2: 'Kopers_afgehaakt', 3: 'Eters_afgehaakt', 4: 'Omzet_kopers', 5: 'Omzet_eters', 6: 'Omzet',7: 'Model'}, axis='columns')

# Eerste rijen van de dataframe laten zien.
display(df4.head())

,Kopers,Eters,Kopers_afgehaakt,Eters_afgehaakt,Omzet_kopers,Omzet_eters,Omzet,Model
0,343,231,70,59,1344.747121,3441.945865,4786.692986,VariantC
1,342,204,71,39,1375.154997,3316.329058,4691.484055,VariantC
2,330,191,63,48,1331.383093,2870.65966,4202.042753,VariantC
3,335,223,72,54,1313.799582,3368.037026,4681.836608,VariantC
4,339,199,81,36,1293.150573,3312.272359,4605.422931,VariantC


## Resultaten

Voor de pastaria is het financiële plaatje het belangrijkt, daarom wordt er gekeken voor welke variant de gemaakte kosten het snelste terugverdiend worden. Verder is er ook gekeken naar welke groep klanten (eters of kopers) meer profijt heeft van deze investering. Als laatste wordt. Als laatste wordt er een 95%-confidence interval berekend voor het aantal dagen dat het duurt om de investering terug te verdienen.

In [6]:
# Terugverdientijd in dagen per variant

# Gemiddeldes van de omzet elke variant van de 200 simulaties 
huidig_omzet = np.mean(df1['Omzet'])
varianta_omzet = np.mean(df2['Omzet'])
variantb_omzet = np.mean(df3['Omzet'])
variantc_omzet = np.mean(df4['Omzet'])

# Verschillen in omzet vergeleken met de Huidige Variant
verschil_a = varianta_omzet - huidig_omzet
verschil_b = variantb_omzet - huidig_omzet
verschil_c = variantc_omzet - huidig_omzet

# Aantal dagen dat het duurt om de gemaakt kosten terug te verdienen
dagen_a = 60000/verschil_a
dagen_b = 120000 / verschil_b
dagen_c = 150000 / verschil_c

print(f'Om de kosten terug te verdienen duurt het voor variant A {dagen_a:.2f} dagen, voor variant B {dagen_b:.2f} dagen en voor variant C {dagen_c:.2f} dagen.')

Om de kosten terug te verdienen duurt het voor variant A 73.08 dagen, voor variant B 60.39 dagen en voor variant C 70.21 dagen.


In [7]:
# Profijt klantengroep

# Huidige Situatie
p_afg_kopers1 = np.mean(df1['Kopers_afgehaakt'])/np.mean(df1['Kopers'])*100
p_afg_eters1 = np.mean(df1['Eters_afgehaakt'])/np.mean(df1['Eters'])*100

# Variant B (beste variant)
p_afg_kopers2 = np.mean(df3['Kopers_afgehaakt'])/np.mean(df3['Kopers'])*100
p_afg_eters2 = np.mean(df3['Eters_afgehaakt'])/np.mean(df3['Eters'])*100

# Afname in procenten
afname_kopers = ((p_afg_kopers2-p_afg_kopers1)/p_afg_kopers1)*100
afname_eters = ((p_afg_eters2-p_afg_eters1)/p_afg_eters1)*100
print(f'In de Huidige Situatie zijn er {p_afg_kopers1:.2f}% van de kopers afgehaakt en in variant B was dit {p_afg_kopers2:.2f}%. Dit is een afname van {-afname_kopers:.2f}%.')
print(f'In de Huidige Situatie zijn er {p_afg_eters1:.2f}% van de eters afgehaakt en in variant B was dit {p_afg_eters2:.2f}%. Dit is een afname van {-afname_eters:.2f}%.')

In de Huidige Situatie zijn er 45.96% van de kopers afgehaakt en in variant B was dit 7.30%. Dit is een afname van 84.13%.
In de Huidige Situatie zijn er 60.91% van de eters afgehaakt en in variant B was dit 29.88%. Dit is een afname van 50.94%.


### 95%-Confidence Interval berekenen

Voor het berekenen van de 95%-CI wordt gebruik gemaakt van de Centrale Limietstelling (CLT). Hierbij kan worden aangenomen dat bij een groot genoeg aantal steekproeven de resultaten (of de verschillen daarvan) bij benadering normaal verdeeld zijn. Omdat, met behulp van Monte Carlo, alle simulaties 200 keer zijn uitgevoerd is de steekproef groot genoeg om aan te nemen dat de CLT hier geldt.

In [8]:
# Verschil in omzet
diff = df3['Omzet'].to_numpy() - df1['Omzet'].to_numpy()

desc = sms.DescrStatsW(diff)

# 95% CI voor het gemiddelde verschil
ci_laag, ci_hoog = desc.tconfint_mean(alpha=0.05)

# CI Omzetten naar dagen
ci_dagen = (120000/ci_hoog, 120000/ci_laag)
print(f"95% CI voor dagen van Variant B:  ({ci_dagen[0]:.2f}, {ci_dagen[1]:.2f})")

95% CI voor dagen van Variant B:  (59.85, 60.95)


## Conclusie

Uit dit onderzoek is gebleken dat de beste oplossing voor de pastaria *variant B* is. De investering van €120.000 wordt terugverdiend in ± 60 dagen, met een 95% Confidence Interval van 59,85 tot 60,95 dagen. Dit betekent dat met 95% zekerheid gesteld mag worden dat de gemiddelde terugverdientijd tussen de 59,85 en 60,95 dagen ligt.

De klantengroep die het meest profijt zal hebben van de veranderingen in *variant B* zijn de kopers. Uit de simulaties blijkt dat er in vergelijking met de *huidige situatie* een afname van **84.13%** is in het aantal kopers dat teleurgesteld naar huis moet (van 45.96% naar 7,30%). Daarnaast zullen ook de eters van deze veranderingen blij worden, voor hen neemt het aantal eters dat teleurgesteld naar huis moet af met **50,94%** (van 60,91% naar 29,88%). 

In *variant B* heb je als koper minder kans om naar huis gestuurd te worden (7,30%) ten opzichte van de eters (29,88%).